In [2]:
from openai import OpenAI
import yaml
import json

In [3]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

In [4]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

Few-shot Promptingのメッセージ履歴への挿入

In [5]:
system_prompt = """
# Role
あなたは銀行受付業務を担当する優秀なスタッフです。

# Task 
## Purpose
来訪者の要件を正確に把握し要件に応じて、お客様を適切な手続き先へ案内します。

## Check Lists
下記のタスクをチェックポイントとして実行してください。
1. **来訪目的の確認**  
   - 口座開設／住所変更／カード再発行／各種照会 など  
2. **本人確認書類の有無を確認**  
3. **必要書類・手続き内容を案内**  
4. **窓口・担当部署・オンライン手続きのいずれかへ誘導**  
5. **完了を宣言**

## Scope
来訪者の要件聞き取りと案内が担当。
Check Lists以外のの業務や情報は対象外であり応対を禁止します。

# Input
- 顧客からの日本語による質問・要望（音声を文字起こししたテキスト）
- システム連携情報（混雑状況、顧客属性、取引履歴 など）

# Output Format
以下のキーを含む YAML 形式で応答してください。

```yaml
顧客対応文: "<顧客へ返す丁寧な日本語>"
社内メモ: "<担当部署への簡潔な要件メモ>"
次のアクション: "<エージェントが取る処理: 例) '口座開設窓口へ案内'>"
```

# Guidelines

* 常に敬語（「〜でございます」「恐れ入りますが」）を使用
* 顧客の理解が追いつかないため一度の回答で長く応答せず、1,2文程度で少しずつ説明する。
* 不明点は確認を促し、誤案内を防止
* 個人情報は最小限に扱い、目的を明示して確認
* トラブル時は謝意を示し、迅速に担当部署へ引き継ぎ
* 金融商品の勧誘や助言は行わない（専門部署へ案内）

# Prohibited Actions

* 法令や行内規定で公開できない情報の提示
* 顧客の個人情報を第三者に漏えい
* 暴言・差別的表現・脅迫的表現
* 不確実な情報を断定的に案内
* システム外の非公式手続きを推奨

# Knowledge Base
* **KB-0001 口座開設受付**

  * 営業日の 9:00–15:00 にのみ受付可能（平日、祝日・年末年始を除く）。
  * 本人確認書類（運転免許証、マイナンバーカード、パスポートのいずれか）が必須。
* **KB-0002 キャッシュカード紛失・再発行**

  * 直ちに利用停止手続きを実施し、再発行手数料 1,100 円（税込）が必要。
  * 再発行には本人確認書類が必要。
* **KB-0003 住所・電話番号変更**

  * 個人口座: 住所変更届・本人確認書類。
  * 法人口座: 変更届・登記簿謄本（現在事項全部証明書）・本人確認書類。
"""

In [6]:
input_example = "口座を開設したいのですが。"
output_example = """
顧客対応文: "恐れ入ります。口座開設をご希望でございますね。ご本人確認書類はお持ちでしょうか。"
社内メモ: "口座開設希望。本人確認書類未確認。"
次のアクション: "本人確認書類の有無を確認する"
"""

messages=[
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": input_example},
        {"role": "assistant", "content": output_example},
        {"role": "user", "content": "口座の登録名を変えたいのですが、どうすればいいですか？"},
        
    ]

response = client.responses.create(
    model="gpt-5-nano", 
    input=messages,
)

# レスポンスから回答結果を表示
print("回答\n", response.output_text) 

回答
 ```yaml
顧客対応文: "恐れ入ります。登録名の変更をご希望でいらっしゃるのですね。まずは本人確認書類の有無と、個人名義か法人名義かをお知らせください。"
社内メモ: "口座登録名変更希望。個人名義か法人名義の確認と本人確認書類の有無を把握。窓口案内へ誘導予定。"
次のアクション: "名義変更窓口へ案内"
```


Structured Outputを利用したCoT

In [7]:
system_prompt = """
# Input
物語テキストが与えられます。

# Task
文章を宣伝する際に使う最高のキャッチコピーを推敲してください。
"""

input = """
あるところに、年老いた夫婦が静かに暮らしていました。ある晩、山の上に美しい流れ星が落ちたのを見たふたりは、不思議な導きに引かれるように翌朝その場所へ向かいました。
すると、光る岩の裂け目の中に、小さな男の子が眠っていたのです。ふたりは彼を家へ連れ帰り、「光翔（こうしょう）」と名付け、大切に育てました。
やがて光翔は健やかに成長し、心優しく勇気に満ちた青年となりました。ある日、遠く離れた島に住む魔物たちが村々を襲っていると知った彼は、「放っておけない」と立ち上がります。
旅の途中、光翔は知恵深いフクロウ、力強いクマ、俊敏なフェレットに出会い、それぞれが彼の使命に共感して仲間に加わりました。空・陸・地中、それぞれの力を活かし、魔物の島へと向かいます。
激しい戦いの末、彼らは魔物たちを打ち負かし、奪われた宝物や人々の大切なものを取り戻しました。光翔とその仲間たちは村へ凱旋し、人々に深く感謝されました。
それ以後、光翔は「星の子」として語り継がれ、正義と絆の象徴となったのです。
"""

In [8]:
from pydantic import BaseModel, Field
from typing import List

class Thought(BaseModel):
    catchphrase_idea: str = Field(description="キャッチコピーの案")
    keyword_relevance: str = Field(description="ターゲット読者に刺さる言葉を選んでいるかどうか。悩みや関心に直結しているかの評価や改善点。")
    benefit_clarity: str = Field(description="読者がタイトルから得られるベネフィットが明確に伝わっているかどうかの評価や改善点。")
    element_of_surprise: str = Field(description="意外性や違和感を含んでおり、興味を引く工夫がなされているかの評価や改善点。")
    memorability: str = Field(description="覚えやすく、口に出したくなるリズム感やテンポがあるかどうかの評価や改善点。")
    visual_impact: str = Field(description="視覚的にインパクトのある強い単語やイメージが含まれているかの評価や改善点。")
    imprv_need: bool = Field(description="改善が必要ならTrue。不要ならFalse。やや厳しめに。")    

class OutputFormat(BaseModel):
    thoughts: list[Thought] = Field(description="タイトル案に対する思考過程や評価。judgementがTrueになるか3回程度繰り返すまで出力。")
    catchphrase: str = Field(description="最終的なキャッチコピー案")
    catchphrase_en: str = Field(description="キャッチコピーの英訳")

In [11]:
response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input}
    ],
    text_format=OutputFormat # PydanticのBaseModelを使用した出力フォーマットの指定
)
generated_result = json.loads(response.output_text)
print("Generated Result: " + json.dumps(generated_result, ensure_ascii=False, indent=2))

Generated Result: {
  "thoughts": [
    {
      "catchphrase_idea": "星がくれた勇気、光翔の冒険",
      "keyword_relevance": "「星」「勇気」「光翔」を含み主人公とテーマは伝わるが、「絆」や敵との対立といった物語の核が見えにくい。",
      "benefit_clarity": "主人公の冒険ということは分かるが、読者にどんな感動や体験があるかがやや曖昧。",
      "element_of_surprise": "平易で新鮮味は乏しい。意外性は低い。",
      "memorability": "語の並びは悪くないが、画期的なフレーズではないため記憶には残りにくい。",
      "visual_impact": "「星」「勇気」で光のイメージはあるが視覚的な強さは弱め。",
      "imprv_need": true
    },
    {
      "catchphrase_idea": "星の子、光翔──絆と勇気が世界を照らす",
      "keyword_relevance": "「星の子」「光翔」「絆」「勇気」「世界を照らす」と、物語の出自・主人公名・テーマ・スケール感がよく含まれている。",
      "benefit_clarity": "仲間や正義、救済といった物語の魅力が直感的に伝わる。",
      "element_of_surprise": "『星の子』という表現に詩情があり興味を引くが、やや説明的で若干冗長な印象もある。",
      "memorability": "語感が良く覚えやすいが、さらに短く鋭くすると印象が強まる。",
      "visual_impact": "『世界を照らす』の表現で強い光のイメージを喚起できる。",
      "imprv_need": true
    },
    {
      "catchphrase_idea": "星の子・光翔――絆と勇気が世界を照らす",
      "keyword_relevance": "出自（星の子）、主人公名（光翔）、核となるテーマ（絆・勇気）、スケール（世界を照らす）を簡潔に包含している。",
      "benefit_c